<a href="https://colab.research.google.com/github/daniolivaglez/03MAIR-Algoritmos-De-Optimizacion/blob/main/AG3/Daniel_Oliva_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3- Actividad Guiada 3 <br>
Nombre: Daniel Oliva González <br>
https://colab.research.google.com/drive/1H4hge994trpr4LVTD5u6Jbx6khRS9Z3H?usp=sharing <br>
https://github.com/daniolivaglez/03MAIR-Algoritmos-De-Optimizacion/blob/main/AG3/Daniel_Oliva_AG3.ipynb

In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

In [2]:
import tsplib95       #Modulo para las instancias del problema del TSP
import random         #Modulo para generar números aleatorios
import math
import copy           #Para copia profunda de estructuras de datos(en python la asignación es por referencia)
from math import e
import numpy as np


Ahora se va a descargar el fichero para realizar la actividad. 

In [3]:
import urllib.request #Hacer llamadas http a paginas de la red

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp", file)


('swiss42.tsp', <http.client.HTTPMessage at 0x7f5df9d32ef0>)

In [4]:
#Carga de datos y generación de objeto problem

problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [5]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
#dir(problem)

15

En la siguiente celda se definirán varias funciones básicas que ayudarán a encontrar soluciones.

In [6]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

El siguiente paso será implementar el algoritmo de búsqueda aleatoria.

In [7]:
def busqueda_aleatoria(problem, N):
  
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = 10e100                          #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  

    
    
solucion = busqueda_aleatoria(problem, 20000)    
 

Mejor solución: [0, 13, 12, 34, 33, 15, 1, 38, 19, 10, 9, 41, 23, 21, 3, 7, 32, 16, 14, 20, 35, 17, 37, 31, 27, 5, 2, 22, 28, 36, 18, 11, 4, 6, 30, 39, 24, 29, 40, 26, 25, 8]
Distancia     : 3703


El próximo paso es crear la función que genera soluciones vecinas. En el caso de la práctica guiada es realizando el cambio de dos posiciones. Se va a introducir una **modificación** de tal forma que se cree una sublista interna a la solución y se invierta el orden.

In [8]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      ## vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  
      
    ############################################### MODIFICACIÓN #######################################################
      vecina = solucion[:i] + list(reversed(solucion[i:j])) + solucion[j:] 
    #####################################################################################################################
      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Inicial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina(solucion)
print("Distancia Solucion Local:", distancia_total(nueva_solucion, problem))

 

Distancia Solucion Inicial: 3703
Distancia Solucion Local: 3460


In [9]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0   #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1     #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)
    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminas. Hemos llegado a un minimo local(según nuestro operador de vencindad)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina
 
 
sol = busqueda_local(problem )  


En la iteracion  36 , la mejor solución encontrada es: [0, 1, 7, 31, 35, 36, 17, 37, 15, 16, 14, 19, 13, 5, 6, 3, 4, 26, 18, 12, 11, 25, 10, 8, 9, 41, 23, 21, 40, 24, 39, 22, 38, 30, 29, 28, 2, 27, 32, 34, 20, 33]
Distancia     : 1404


Ahora se va a simular la forja de metales con la técnica de recocido simulado.

In [12]:
#Generador de 1 solucion vecina 2-opt aleatoria (intercambiar 2 nodos)
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
 
genera_vecina_aleatorio(solucion)

[0,
 13,
 12,
 34,
 33,
 15,
 1,
 38,
 19,
 10,
 9,
 40,
 23,
 21,
 3,
 7,
 32,
 16,
 14,
 20,
 35,
 17,
 37,
 31,
 27,
 5,
 2,
 22,
 28,
 36,
 18,
 11,
 4,
 6,
 30,
 39,
 24,
 29,
 41,
 26,
 25,
 8]

Para esta simulación se necesita la función de probabilidad y otra que permita bajar la temperatura

In [10]:
#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

Ahora se implementará la función de recocido simulado:

In [13]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
  ## antes de nada se establecen las soluciones iniciales
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  ## se inicializan las variables que guardarán las soluciones finales
  mejor_solucion = []
  mejor_distancia = 10e100
  
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = copy.deepcopy(vecina)
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 20, 33, 12, 11, 18, 4, 27, 2, 10, 25, 29, 30, 34, 32, 28, 3, 1, 26, 5, 13, 19, 16, 15, 14, 6, 8, 9, 39, 38, 22, 23, 41, 40, 24, 21, 7, 37, 31, 36, 35, 17]
con una distancia total de 2155
